# Wakeword Detection Training System

## GPU-Accelerated Wakeword Training with CNN+LSTM Architecture

**System Status**: ✅ GPU Ready | RTX 3060 Ti | PyTorch 2.0.1 + CUDA 11.8

In [ ]:
# System Setup and Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import librosa
import soundfile as sf
import os
import glob
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
import subprocess
import sys
warnings.filterwarnings('ignore')

# GPU Configuration and Diagnostic
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔍 System Diagnostic:")
print(f"   PyTorch version: {torch.__version__}")
print(f"   Using device: {device}")
print(f"   GPU Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"   CUDA Version: {torch.version.cuda}")
else:
    print("   ⚠️  GPU not detected - running on CPU")
    print("   🔧 GPU Fix Suggestions:")
    print("      1. Install NVIDIA drivers: sudo apt install nvidia-driver-535")
    print("      2. Install CUDA toolkit")
    print("      3. Restart WSL: wsl --shutdown (in Windows CMD)")
    print("      4. Run: pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")

## Configuration

In [ ]:
# Audio Configuration
class AudioConfig:
    SAMPLE_RATE = 16000
    DURATION = 1.7  # seconds
    N_MELS = 80
    N_FFT = 2048
    HOP_LENGTH = 512
    WIN_LENGTH = 2048
    FMIN = 0
    FMAX = 8000
    
# Model Configuration - Enhanced for better training
class ModelConfig:
    HIDDEN_SIZE = 256  # Increased capacity
    NUM_LAYERS = 2
    DROPOUT = 0.6  # Increased dropout for regularization
    NUM_CLASSES = 2  # wakeword vs negative
    
# Training Configuration - Fixed for better convergence
class TrainingConfig:
    BATCH_SIZE = 32  # Reduced for better gradient updates
    LEARNING_RATE = 0.0001  # Reduced for stable training
    EPOCHS = 100
    VALIDATION_SPLIT = 0.2
    TEST_SPLIT = 0.1
    
# Data Augmentation Configuration - Enhanced for better generalization
class AugmentationConfig:
    AUGMENTATION_PROB = 0.85  # Increased augmentation
    NOISE_FACTOR = 0.15  # More noise variation
    TIME_SHIFT_MAX = 0.3  # More time variation
    PITCH_SHIFT_MAX = 1.5  # More pitch variation
    SPEED_CHANGE_MIN = 0.9
    SPEED_CHANGE_MAX = 1.1

print("Configuration loaded successfully!")
print("🔧 Enhanced configuration for better training convergence")
print(f"   Learning Rate: {TrainingConfig.LEARNING_RATE}")
print(f"   Batch Size: {TrainingConfig.BATCH_SIZE}")
print(f"   Dropout: {ModelConfig.DROPOUT}")
print(f"   Augmentation: {AugmentationConfig.AUGMENTATION_PROB}")

## Audio Processing Class

In [ ]:
class AudioProcessor:
    """Handle all audio processing tasks"""
    
    def __init__(self, config=AudioConfig):
        self.config = config
        
    def load_audio(self, file_path):
        """Load audio file and return as numpy array"""
        try:
            audio, sr = librosa.load(file_path, sr=self.config.SAMPLE_RATE)
            return audio
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            return None
    
    def normalize_audio(self, audio):
        """Normalize audio to [-1, 1] range"""
        if len(audio) == 0:
            return audio
        return audio / np.max(np.abs(audio))
    
    def pad_or_truncate(self, audio, target_length):
        """Pad or truncate audio to target length"""
        if len(audio) > target_length:
            # Truncate
            start_idx = random.randint(0, len(audio) - target_length)
            return audio[start_idx:start_idx + target_length]
        else:
            # Pad with zeros
            return np.pad(audio, (0, target_length - len(audio)), mode='constant')
    
    def audio_to_mel(self, audio):
        """Convert audio to mel-spectrogram"""
        if len(audio) == 0:
            return np.zeros((self.config.N_MELS, int(self.config.SAMPLE_RATE * self.config.DURATION / self.config.HOP_LENGTH) + 1))
        
        mel_spec = librosa.feature.melspectrogram(
            y=audio,
            sr=self.config.SAMPLE_RATE,
            n_mels=self.config.N_MELS,
            n_fft=self.config.N_FFT,
            hop_length=self.config.HOP_LENGTH,
            win_length=self.config.WIN_LENGTH,
            fmin=self.config.FMIN,
            fmax=self.config.FMAX
        )
        
        # Convert to log scale
        mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
        return mel_spec
    
    def augment_audio(self, audio, config=AugmentationConfig):
        """Apply data augmentation to audio"""
        augmented_audio = audio.copy()
        
        # Time shift
        if random.random() < config.AUGMENTATION_PROB:
            shift_amount = int(random.uniform(-config.TIME_SHIFT_MAX, config.TIME_SHIFT_MAX) * self.config.SAMPLE_RATE)
            augmented_audio = np.roll(augmented_audio, shift_amount)
        
        # Pitch shift
        if random.random() < config.AUGMENTATION_PROB:
            n_steps = random.uniform(-config.PITCH_SHIFT_MAX, config.PITCH_SHIFT_MAX)
            augmented_audio = librosa.effects.pitch_shift(y=augmented_audio, sr=self.config.SAMPLE_RATE, n_steps=n_steps)
        
        # Speed change
        if random.random() < config.AUGMENTATION_PROB:
            speed_factor = random.uniform(config.SPEED_CHANGE_MIN, config.SPEED_CHANGE_MAX)
            augmented_audio = librosa.effects.time_stretch(y=augmented_audio, rate=speed_factor)
            
            # Pad or truncate to maintain original length
            augmented_audio = self.pad_or_truncate(augmented_audio, len(audio))
        
        # Add background noise
        if random.random() < config.AUGMENTATION_PROB:
            noise = np.random.normal(0, config.NOISE_FACTOR, len(augmented_audio))
            augmented_audio = augmented_audio + noise
        
        return augmented_audio
    
    def process_audio_file(self, file_path, augment=False):
        """Process a single audio file"""
        # Load audio
        audio = self.load_audio(file_path)
        if audio is None:
            return None
        
        # Normalize
        audio = self.normalize_audio(audio)
        
        # Pad/truncate to target length
        target_length = int(self.config.SAMPLE_RATE * self.config.DURATION)
        audio = self.pad_or_truncate(audio, target_length)
        
        # Apply augmentation if requested
        if augment:
            audio = self.augment_audio(audio)
        
        # Convert to mel-spectrogram
        mel_spec = self.audio_to_mel(audio)
        
        return mel_spec

# Test the audio processor
processor = AudioProcessor()
print("AudioProcessor created successfully!")

## Neural Network Model

In [ ]:
class WakewordModel(nn.Module):
    """CNN+LSTM model for wakeword detection"""
    
    def __init__(self, config=ModelConfig, audio_config=AudioConfig):
        super(WakewordModel, self).__init__()
        self.config = config
        self.audio_config = audio_config
        
        # Calculate CNN output size
        self.mel_height = audio_config.N_MELS
        self.mel_width = int(audio_config.SAMPLE_RATE * audio_config.DURATION / audio_config.HOP_LENGTH) + 1
        
        # CNN layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Calculate CNN output size
        self.cnn_output_size = 128  # After adaptive pooling
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=self.cnn_output_size,
            hidden_size=config.HIDDEN_SIZE,
            num_layers=config.NUM_LAYERS,
            batch_first=True,
            dropout=config.DROPOUT if config.NUM_LAYERS > 1 else 0
        )
        
        # Dropout
        self.dropout = nn.Dropout(config.DROPOUT)
        
        # Final classification layer
        self.fc = nn.Linear(config.HIDDEN_SIZE, config.NUM_CLASSES)
        
    def forward(self, x):
        # Input shape: (batch, 1, mel_height, mel_width)
        batch_size = x.size(0)
        
        # CNN feature extraction
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        
        # Reshape for LSTM
        x = x.view(batch_size, -1)  # (batch, cnn_output_size)
        
        # Add sequence dimension
        x = x.unsqueeze(1)  # (batch, seq_len=1, features)
        
        # LSTM processing
        lstm_out, (h_n, c_n) = self.lstm(x)
        
        # Take the last output
        x = lstm_out[:, -1, :]  # (batch, hidden_size)
        
        # Dropout
        x = self.dropout(x)
        
        # Final classification
        x = self.fc(x)
        
        return x

# Create model and move to device
model = WakewordModel().to(device)
print(f"Model created and moved to {device}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

# Test model with dummy input
dummy_input = torch.randn(1, 1, AudioConfig.N_MELS, 31).to(device)  # (batch, channels, height, width)
with torch.no_grad():
    output = model(dummy_input)
print(f"Dummy output shape: {output.shape}")

## Dataset Class

In [ ]:
# Enhanced Dataset Class with Background Noise Support

class EnhancedWakewordDataset(Dataset):
    """Enhanced dataset with background noise mixing and balanced sampling"""
    
    def __init__(self, wakeword_files, hard_negative_files, random_negative_files, 
                 background_files, processor, augment=False, 
                 background_mix_prob=0.7, snr_range=(0, 20)):
        """
        Args:
            wakeword_files: List of positive wakeword audio files
            hard_negative_files: List of hard negative files (phonetically similar)
            random_negative_files: List of random negative files
            background_files: List of background noise files (66 hours)
            processor: AudioProcessor instance
            augment: Whether to apply augmentation
            background_mix_prob: Probability of mixing with background noise
            snr_range: SNR range in dB for mixing
        """
        self.processor = processor
        self.augment = augment
        self.background_mix_prob = background_mix_prob
        self.snr_range = snr_range
        
        # Store file lists separately for balanced sampling
        self.wakeword_files = wakeword_files
        self.hard_negative_files = hard_negative_files
        self.random_negative_files = random_negative_files
        self.background_files = background_files
        
        # Pre-load background noise segments for efficient mixing
        self.background_cache = self._cache_background_segments()
        
        # Calculate balanced dataset composition
        self._create_balanced_dataset()
        
    def _cache_background_segments(self, max_cache_size=100):
        """Pre-load and cache background noise segments"""
        print("🎵 Caching background noise segments...")
        cache = []
        
        for i, bg_file in enumerate(self.background_files[:max_cache_size]):
            try:
                audio = self.processor.load_audio(bg_file)
                if audio is not None and len(audio) > 0:
                    # Normalize background audio
                    audio = audio / (np.max(np.abs(audio)) + 1e-8)
                    cache.append(audio)
            except Exception as e:
                print(f"Warning: Could not load background file {bg_file}: {e}")
            
            if (i + 1) % 20 == 0:
                print(f"   Cached {i + 1}/{min(max_cache_size, len(self.background_files))} background files")
        
        print(f"✅ Cached {len(cache)} background noise segments")
        return cache
    
    def _create_balanced_dataset(self):
        """Create balanced dataset with proper ratios"""
        # Target ratios: 1 positive : 4.5 hard_neg : 8.75 random_neg : 10 background
        
        n_wakeword = len(self.wakeword_files)
        
        # Calculate other categories based on ratios
        n_hard_neg = min(len(self.hard_negative_files), int(n_wakeword * 4.5))
        n_random_neg = min(len(self.random_negative_files), int(n_wakeword * 8.75))
        n_background_pure = min(len(self.background_files), int(n_wakeword * 10))
        
        # Sample files for each category
        import random
        random.seed(42)  # For reproducibility
        
        sampled_hard_neg = random.sample(self.hard_negative_files, n_hard_neg) if n_hard_neg > 0 else []
        sampled_random_neg = random.sample(self.random_negative_files, n_random_neg) if n_random_neg > 0 else []
        sampled_background = random.sample(self.background_files, n_background_pure) if n_background_pure > 0 else []
        
        # Combine all files with their labels
        self.files = []
        self.labels = []
        self.categories = []  # Track category for analysis
        
        # Add wakeword samples (label=1)
        for f in self.wakeword_files:
            self.files.append(f)
            self.labels.append(1)
            self.categories.append('wakeword')
        
        # Add hard negatives (label=0)
        for f in sampled_hard_neg:
            self.files.append(f)
            self.labels.append(0)
            self.categories.append('hard_negative')
        
        # Add random negatives (label=0)
        for f in sampled_random_neg:
            self.files.append(f)
            self.labels.append(0)
            self.categories.append('random_negative')
        
        # Add pure background noise (label=0)
        for f in sampled_background:
            self.files.append(f)
            self.labels.append(0)
            self.categories.append('background')
        
        # Shuffle the dataset
        indices = list(range(len(self.files)))
        random.shuffle(indices)
        self.files = [self.files[i] for i in indices]
        self.labels = [self.labels[i] for i in indices]
        self.categories = [self.categories[i] for i in indices]
        
        # Print dataset statistics
        print(f"\n📊 Enhanced Dataset Statistics:")
        print(f"   Wakeword samples: {n_wakeword}")
        print(f"   Hard negatives: {n_hard_neg}")
        print(f"   Random negatives: {n_random_neg}")
        print(f"   Background noise: {n_background_pure}")
        print(f"   Total samples: {len(self.files)}")
        print(f"   Positive ratio: 1:{(len(self.files) - n_wakeword) / n_wakeword:.1f}")
    
    def _mix_with_background(self, audio, target_snr_db=None):
        """Mix audio with random background noise at specified SNR"""
        if len(self.background_cache) == 0:
            return audio
        
        # Select random background
        bg_audio = random.choice(self.background_cache)
        
        # Extract segment from background
        target_len = len(audio)
        if len(bg_audio) > target_len:
            start_idx = random.randint(0, len(bg_audio) - target_len)
            bg_segment = bg_audio[start_idx:start_idx + target_len]
        else:
            # Tile background if it's shorter
            bg_segment = np.tile(bg_audio, (target_len // len(bg_audio) + 1))[:target_len]
        
        # Calculate SNR if not specified
        if target_snr_db is None:
            target_snr_db = random.uniform(self.snr_range[0], self.snr_range[1])
        
        # Calculate mixing weights based on SNR
        signal_power = np.mean(audio ** 2)
        noise_power = np.mean(bg_segment ** 2)
        
        if noise_power > 0:
            current_snr = 10 * np.log10(signal_power / noise_power)
            noise_scale = np.sqrt(signal_power / (10 ** (target_snr_db / 10)) / noise_power)
            mixed = audio + noise_scale * bg_segment
        else:
            mixed = audio
        
        # Normalize to prevent clipping
        max_val = np.max(np.abs(mixed))
        if max_val > 0:
            mixed = mixed / max_val * 0.95
        
        return mixed
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        file_path = self.files[idx]
        label = self.labels[idx]
        category = self.categories[idx]
        
        # Load and process audio
        audio = self.processor.load_audio(file_path)
        
        if audio is None:
            # Return zeros if loading failed
            mel_spec = np.zeros((self.processor.config.N_MELS, 31))
        else:
            # Normalize audio
            audio = self.processor.normalize_audio(audio)
            
            # Pad/truncate to target length
            target_length = int(self.processor.config.SAMPLE_RATE * self.processor.config.DURATION)
            audio = self.processor.pad_or_truncate(audio, target_length)
            
            # Apply augmentation if training
            if self.augment:
                audio = self.processor.augment_audio(audio)
            
            # Mix with background noise (except for pure background samples)
            if category != 'background' and random.random() < self.background_mix_prob:
                audio = self._mix_with_background(audio)
            
            # Convert to mel-spectrogram
            mel_spec = self.processor.audio_to_mel(audio)
        
        # Convert to tensor
        mel_tensor = torch.FloatTensor(mel_spec).unsqueeze(0)
        label_tensor = torch.LongTensor([label])
        
        return mel_tensor, label_tensor


## Training Class

In [ ]:
class WakewordTrainer:
    """Enhanced training class with learning rate scheduling"""
    
    def __init__(self, model, device, config=TrainingConfig):
        self.model = model
        self.device = device
        self.config = config
        
        # Loss function and optimizer
        self.criterion = nn.CrossEntropyLoss().to(device)
        self.optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE, weight_decay=1e-5)
        
        # Learning rate scheduler
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='max', factor=0.5, patience=5
        )
        
        # Training history
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []
        
        # Early stopping
        self.patience = 10
        self.best_val_acc = 0.0
        self.epochs_no_improve = 0
        
    def train_epoch(self, train_loader):
        """Train for one epoch with gradient clipping"""
        self.model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for batch_idx, (data, target) in enumerate(tqdm(train_loader, desc="Training")):
            data, target = data.to(self.device), target.to(self.device).squeeze()
            
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = self.criterion(output, target)
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            
            loss.backward()
            self.optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100. * correct / total
        
        return epoch_loss, epoch_acc
    
    def validate(self, val_loader):
        """Validate the model"""
        self.model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for data, target in tqdm(val_loader, desc="Validating"):
                data, target = data.to(self.device), target.to(self.device).squeeze()
                output = self.model(data)
                loss = self.criterion(output, target)
                
                running_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
        
        epoch_loss = running_loss / len(val_loader)
        epoch_acc = 100. * correct / total
        
        return epoch_loss, epoch_acc
    
    def train(self, train_loader, val_loader, epochs):
        """Complete training loop with early stopping"""
        print(f"Starting training for {epochs} epochs...")
        print(f"Using device: {self.device}")
        print(f"Learning rate: {self.config.LEARNING_RATE}")
        print(f"Batch size: {self.config.BATCH_SIZE}")
        
        self.best_val_acc = 0.0
        self.epochs_no_improve = 0
        
        for epoch in range(epochs):
            print(f"\nEpoch {epoch+1}/{epochs}")
            
            # GPU Memory monitoring
            if torch.cuda.is_available():
                gpu_memory = torch.cuda.memory_allocated() / 1e6
                gpu_reserved = torch.cuda.memory_reserved() / 1e6
                print(f"GPU Memory: {gpu_memory:.1f}MB allocated, {gpu_reserved:.1f}MB reserved")
            else:
                print("GPU Memory: Not available (using CPU)")
            
            # Training
            train_loss, train_acc = self.train_epoch(train_loader)
            
            # Validation
            val_loss, val_acc = self.validate(val_loader)
            
            # Store metrics
            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            self.train_accuracies.append(train_acc)
            self.val_accuracies.append(val_acc)
            
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
            print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
            
            # Learning rate scheduling
            self.scheduler.step(val_acc)
            
            # Early stopping check
            if val_acc > self.best_val_acc:
                self.best_val_acc = val_acc
                self.epochs_no_improve = 0
                
                # Save best model
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'val_acc': val_acc,
                    'train_acc': train_acc,
                    'train_loss': train_loss,
                    'val_loss': val_loss,
                }, 'best_wakeword_model.pth')
                print(f"🎉 New best model saved! Validation accuracy: {val_acc:.2f}%")
            else:
                self.epochs_no_improve += 1
                
            # Early stopping
            if self.epochs_no_improve >= self.patience:
                print(f"\n⏹️  Early stopping triggered! No improvement for {self.patience} epochs.")
                print(f"Best validation accuracy: {self.best_val_acc:.2f}%")
                break
        
        print(f"\n🎉 Training completed!")
        print(f"Best validation accuracy: {self.best_val_acc:.2f}%")
        print(f"Total epochs trained: {epoch + 1}")
        return self.best_val_acc
    
    def plot_training_history(self):
        """Plot training history"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss plot
        ax1.plot(self.train_losses, label='Train Loss', linewidth=2)
        ax1.plot(self.val_losses, label='Val Loss', linewidth=2)
        ax1.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Accuracy plot
        ax2.plot(self.train_accuracies, label='Train Accuracy', linewidth=2)
        ax2.plot(self.val_accuracies, label='Val Accuracy', linewidth=2)
        ax2.set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy (%)')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Create trainer instance
trainer = WakewordTrainer(model, device)
print("Enhanced trainer created successfully!")
print("🔧 Added: Learning rate scheduling, gradient clipping, early stopping")

## Data Loading and Preparation

In [ ]:
# Modified Data Loading Section
def prepare_enhanced_datasets(wakeword_dir='positive_dataset',
                             negative_dir='negative_dataset',
                             background_dir='background_noise',
                             hard_negative_dir='hard_negatives'):
    """
    Prepare datasets with proper categorization
    
    Directory structure expected:
    - positive_dataset/: Your positive samples
    - negative_dataset/: General negative samples
    - hard_negatives/: Phonetically similar negatives (if separate)
    - background_noise/: 66 hours of background recordings
    """

## Model Training

In [ ]:
# Update the training cell to use enhanced dataset
def create_enhanced_dataloaders(data_splits, processor, batch_size=16):
    """Create DataLoaders with the enhanced dataset"""
    
    # Unpack splits
    wake_train, hard_train, rand_train, bg_train = data_splits['train']
    wake_val, hard_val, rand_val, bg_val = data_splits['val']
    wake_test, hard_test, rand_test, bg_test = data_splits['test']
    
    # Create datasets
    train_dataset = EnhancedWakewordDataset(
        wakeword_files=wake_train,
        hard_negative_files=hard_train,
        random_negative_files=rand_train,
        background_files=bg_train,
        processor=processor,
        augment=True,
        background_mix_prob=0.7,
        snr_range=(0, 20)
    )
    
    val_dataset = EnhancedWakewordDataset(
        wakeword_files=wake_val,
        hard_negative_files=hard_val,
        random_negative_files=rand_val,
        background_files=bg_val,
        processor=processor,
        augment=False,
        background_mix_prob=0.5,  # Less mixing for validation
        snr_range=(5, 15)  # More conservative SNR for validation
    )
    
    test_dataset = EnhancedWakewordDataset(
        wakeword_files=wake_test,
        hard_negative_files=hard_test,
        random_negative_files=rand_test,
        background_files=bg_test,
        processor=processor,
        augment=False,
        background_mix_prob=0.5,
        snr_range=(5, 15)
    )
    
    # Create DataLoaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    return train_loader, val_loader, test_loader


# Usage in your notebook:
if __name__ == "__main__":
    # Prepare datasets
    data_splits = prepare_enhanced_datasets()
    
    # Create processor (assuming it's already defined)
    processor = AudioProcessor()
    
    # Create DataLoaders
    train_loader, val_loader, test_loader = create_enhanced_dataloaders(
        data_splits, 
        processor, 
        batch_size=TrainingConfig.BATCH_SIZE
    )
    
    print("\n🚀 Ready for training with background noise integration!")
    print(f"   Background noise mixing probability: 70% for training")
    print(f"   SNR range: 0-20 dB for training, 5-15 dB for validation")
    print(f"   Total training batches: {len(train_loader)}")
    print(f"   Total validation batches: {len(val_loader)}")

## Model Evaluation

In [ ]:
# Load best model for evaluation
if os.path.exists('best_wakeword_model.pth'):
    checkpoint = torch.load('best_wakeword_model.pth', map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"✅ Best model loaded (epoch {checkpoint['epoch'] + 1}, val_acc: {checkpoint['val_acc']:.2f}%)")
    
    # Evaluate on test set
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc="Evaluating"):
            data, target = data.to(device), target.to(device).squeeze()
            output = model(data)
            _, predicted = torch.max(output, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(target.cpu().numpy())
    
    # Calculate metrics
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    print(f"\n📊 Test Set Performance:")
    print(f"   Accuracy: {accuracy:.4f}")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall: {recall:.4f}")
    print(f"   F1-Score: {f1:.4f}")
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Negative', 'Wakeword'], 
                yticklabels=['Negative', 'Wakeword'])
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    # Classification report
    print("\n📋 Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=['Negative', 'Wakeword']))
    
else:
    print("❌ No trained model found. Please run the training cells first.")

## Model Inference

In [ ]:
def predict_wakeword(audio_file_path, model, processor, device, threshold=0.8):
    """Predict if audio contains wakeword"""
    model.eval()
    
    # Process audio file
    mel_spec = processor.process_audio_file(audio_file_path, augment=False)
    
    if mel_spec is None:
        print(f"Error processing audio file: {audio_file_path}")
        return False, 0.0
    
    # Convert to tensor and add batch dimension
    mel_tensor = torch.FloatTensor(mel_spec).unsqueeze(0).unsqueeze(0).to(device)
    
    # Make prediction
    with torch.no_grad():
        output = model(mel_tensor)
        probabilities = torch.softmax(output, dim=1)
        wakeword_prob = probabilities[0][1].item()
        
    is_wakeword = wakeword_prob >= threshold
    
    return is_wakeword, wakeword_prob

# Test prediction function
if os.path.exists('best_wakeword_model.pth'):
    print("\n🎯 Wakeword Detection System Ready!")
    print("You can now use the predict_wakeword function for real-time detection.")
    print("\nExample usage:")
    print("is_wakeword, confidence = predict_wakeword('./test_files/3.wav', model, processor, device)")
    print(f"Result: {{'wakeword_detected': is_wakeword, 'confidence': confidence:.2f}}")
    
    # Create a simple test function
    def test_audio_file(file_path):
        if os.path.exists(file_path):
            is_wakeword, confidence = predict_wakeword(file_path, model, processor, device)
            print(f"\n📁 Testing: {file_path}")
            print(f"   Wakeword detected: {is_wakeword}")
            print(f"   Confidence: {confidence:.2f}")
            print(f"   Threshold: 0.80")
        else:
            print(f"\n❌ File not found: {file_path}")
    
    # Direct test with ./test_files/1.wav
    test_audio_file("./test_files/3.wav")
else:
    print("❌ Model not trained yet. Please run training cells first.")


## Save Final Model

In [ ]:
# Save the complete model for deployment
if os.path.exists('best_wakeword_model.pth'):
    # Create a complete deployment package
    deployment_package = {
        'model_state_dict': model.state_dict(),
        'model_config': {
            'HIDDEN_SIZE': ModelConfig.HIDDEN_SIZE,
            'NUM_LAYERS': ModelConfig.NUM_LAYERS,
            'DROPOUT': ModelConfig.DROPOUT,
            'NUM_CLASSES': ModelConfig.NUM_CLASSES
        },
        'audio_config': {
            'SAMPLE_RATE': AudioConfig.SAMPLE_RATE,
            'DURATION': AudioConfig.DURATION,
            'N_MELS': AudioConfig.N_MELS,
            'N_FFT': AudioConfig.N_FFT,
            'HOP_LENGTH': AudioConfig.HOP_LENGTH,
            'FMIN': AudioConfig.FMIN,
            'FMAX': AudioConfig.FMAX
        },
        'training_info': {
            'best_val_accuracy': checkpoint.get('val_acc', 0),
            'epoch': checkpoint.get('epoch', 0) + 1,
            'device': str(device)
        },
        'classes': ['negative', 'wakeword']
    }
    
    # Save deployment package
    torch.save(deployment_package, 'wakeword_deployment_model.pth')
    print("✅ Deployment model saved as 'wakeword_deployment_model.pth'")
    
    # Save model architecture for reference
    with open('model_architecture.txt', 'w') as f:
        f.write("Wakeword Detection Model Architecture\n")
        f.write("================================\n\n")
        f.write("Model Type: CNN + LSTM\n")
        f.write(f"Input Shape: (1, {AudioConfig.N_MELS}, 31)\n")
        f.write(f"Hidden Size: {ModelConfig.HIDDEN_SIZE}\n")
        f.write(f"Number of Layers: {ModelConfig.NUM_LAYERS}\n")
        f.write(f"Dropout: {ModelConfig.DROPOUT}\n")
        f.write(f"Number of Classes: {ModelConfig.NUM_CLASSES}\n")
        f.write(f"Parameters: {sum(p.numel() for p in model.parameters()):,}\n")
        f.write(f"Device: {device}\n")
    
    print("✅ Model architecture saved as 'model_architecture.txt'")
    
    print("\n🎉 Model deployment package ready!")
    print("Files created:")
    print("   - wakeword_deployment_model.pth (complete model)")
    print("   - model_architecture.txt (model specs)")
    print("   - best_wakeword_model.pth (training checkpoint)")
    
else:
    print("❌ No trained model found to save.")

## System Summary

In [ ]:
print(f"\n🚀 Next Steps:")
print(f"   1. Add your wakeword recordings to positive_dataset/")
print(f"   2. Add negative samples to negative_dataset/")
print(f"   3. Add background noise to background_noise/")
print(f"   4. Run the training cells above")
print(f"   5. Use the trained model for wakeword detection")